In [1]:
import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
path=os.getcwd()
path

'C:\\Users\\jayam\\Downloads\\7879phd'

In [58]:
train=pd.read_csv(path +"\\train_finaldata.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])

In [59]:
test=pd.read_csv(path +"\\test_finaldata.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])

In [1]:
def split_df(df, col, prefix): ### function for spliting files
    df[col]=df[col].astype("category")
    for l in list(df[col].cat.categories):
        df_fname = prefix + "_" + str(l) + ".csv"
        df[df[col] == l].to_csv(df_fname,index=False)

In [63]:
split_df(test,"city","test")# splitting test file

In [62]:
split_df(train,"city","train")# splitting train file

In [3]:
from statsmodels.tsa.stattools import adfuller
def med_city_dict(df,meds):# function  for storing the medicine list and its pvalues
    mydict={} 
    for m in meds:
        mydict[m]=adfuller(df[df["city_medicine"]==m]["sales"])[1]
    df1=pd.DataFrame(mydict.items(),columns=["city_medicine","p-value"])
    return df1

In [26]:
def read_file(files):# function  for reading the splitted files and then returns p-values
    concatp=pd.DataFrame()
    for file in files:
        file=str(file)
        train=pd.read_csv("train_"+file+".csv")
        train["discounted"]=train["discounted"].fillna(0)
        df=pd.DataFrame(train["city_medicine"].value_counts())
        df.columns=["values"]
        city_med_nopval=list(df[df["values"]>12].index)
        length=len(city_med_nopval)
        #print(length)
        #print(df.index[:length])
        pval=med_city_dict(train,list(df.index[:length]))
        concatp=pd.concat([concatp,pval])
    return concatp

In [31]:
pvalues=read_file(list(range(1,11)))# storing pvalues

In [47]:
city_med_df=pvalues["city_medicine"].str.split("_",expand=True)# splitting city and medicine and storing p values
pvalues["city"]=city_med_df[0]
pvalues["medicine"]=city_med_df[1]
pvalues.to_csv("pvalues.csv",index=False)

In [53]:
len(pvalues[pvalues["p-value"]>0.01])

5693

- So data is not stationary for 5693 combinations of city and medicine considering at 99% confidence intervals

In [55]:
len(pvalues[pvalues["p-value"]>0.05])

3063

- So data is not stationary for 3063 combinations of city and medicine considering at 95% confidence intervals